# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

In [2]:
#!cat Employee_Relationships.txt

Andy	Claude	0
Andy	Frida	20
Andy	Georgia	-10
Andy	Joan	30
Andy	Lee	-10
Andy	Pablo	-10
Andy	Vincent	20
Claude	Frida	0
Claude	Georgia	90
Claude	Joan	0
Claude	Lee	0
Claude	Pablo	10
Claude	Vincent	0
Frida	Georgia	0
Frida	Joan	0
Frida	Lee	0
Frida	Pablo	50
Frida	Vincent	60
Georgia	Joan	0
Georgia	Lee	10
Georgia	Pablo	0
Georgia	Vincent	0
Joan	Lee	70
Joan	Pablo	0
Joan	Vincent	10
Lee	Pablo	0
Lee	Vincent	0
Pablo	Vincent	-20


In [3]:
#!cat Employee_Movie_Choices.txt

#Employee	Movie
Andy	Anaconda
Andy	Mean Girls
Andy	The Matrix
Claude	Anaconda
Claude	Monty Python and the Holy Grail
Claude	Snakes on a Plane
Frida	The Matrix
Frida	The Shawshank Redemption
Frida	The Social Network
Georgia	Anaconda
Georgia	Monty Python and the Holy Grail
Georgia	Snakes on a Plane
Joan	Forrest Gump
Joan	Kung Fu Panda
Joan	Mean Girls
Lee	Forrest Gump
Lee	Kung Fu Panda
Lee	Mean Girls
Pablo	The Dark Knight
Pablo	The Matrix
Pablo	The Shawshank Redemption
Vincent	The Godfather
Vincent	The Shawshank Redemption
Vincent	The Social Network


### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [61]:
def answer_one():

    G_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t', skiprows=1,
                   names=['Employee', 'Movie'])
    G5 = nx.from_pandas_dataframe(G_df, 'Employee', 'Movie') 
    return G5

In [62]:
#G=answer_one()
#G.edges()

[('Andy', 'Anaconda'),
 ('Andy', 'Mean Girls'),
 ('Andy', 'The Matrix'),
 ('Anaconda', 'Claude'),
 ('Anaconda', 'Georgia'),
 ('Mean Girls', 'Joan'),
 ('Mean Girls', 'Lee'),
 ('The Matrix', 'Frida'),
 ('The Matrix', 'Pablo'),
 ('Claude', 'Monty Python and the Holy Grail'),
 ('Claude', 'Snakes on a Plane'),
 ('Monty Python and the Holy Grail', 'Georgia'),
 ('Snakes on a Plane', 'Georgia'),
 ('Frida', 'The Shawshank Redemption'),
 ('Frida', 'The Social Network'),
 ('The Shawshank Redemption', 'Pablo'),
 ('The Shawshank Redemption', 'Vincent'),
 ('The Social Network', 'Vincent'),
 ('Joan', 'Forrest Gump'),
 ('Joan', 'Kung Fu Panda'),
 ('Forrest Gump', 'Lee'),
 ('Kung Fu Panda', 'Lee'),
 ('Pablo', 'The Dark Knight'),
 ('Vincent', 'The Godfather')]

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [54]:
def answer_two():
    G=answer_one()
    #G_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t', skiprows=1, names=['Employee', 'Movie'])
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type='employee')
        else: G.add_node(node, type='movie')
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [ ]:
def answer_three():
        
    G=answer_two()
    P=bipartite.weighted_projected_graph(G, employees)
    return P

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [105]:
def answer_four():
    G=answer_two()
    P=bipartite.weighted_projected_graph(G, employees)   
    Plist=[]
    for edge in P.edges(data=True):
        if edge[0]<edge[1]:
            Plist.append(((edge[0], edge[1]), edge[2]['weight']))
        else: Plist.append(((edge[1], edge[0]), edge[2]['weight']))# Your Code Here
    dfdf=pd.DataFrame(Plist, columns=['pairs','score'])
    dfdf.set_index('pairs', inplace=True)
    rel_df = pd.read_csv('Employee_Relationships.txt', sep='\t', skiprows=1,
                   names=['E1', 'E2', 'rel_score'])
    rel_df['pairs']=rel_df.apply(lambda x: (x[0], x[1]), axis=1)
    del rel_df['E1']
    del rel_df['E2']
    rel_df.set_index('pairs', inplace=True)
    df=rel_df.merge(dfdf, how='outer', left_index=True, right_index=True)
    df.fillna(value=0, inplace=True)
    return df.corr()['rel_score']['score']

In [106]:
answer_four()

0.78839622217334759